<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3599c9bd6cec0bd1036aeca5046f86afac91a7eb8ac960f8f631e0763131f4f8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 10:09:53
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: -14.80 K (-0.11%)
Current PnL: -28.41 L (-18.35%)
CY Booked + Current PnL: -14.12 L (-9.12%)
-------------------
Total profit:  1.32 L
Total loss:  -29.73 L
-------------------
Total Booked + Current PnL: 12.86 L (10.06%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.22%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.04 L (66.52%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 5.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.15,-16.61,20.03,0.09,16832.0,-16741.0,84036.0,95.09,43.0,M-SC,2.37,253.0,-0.99,0.62,10.41,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.88,-11.57,13.11,0.02,20236.0,-20204.0,154355.0,135.49,55.0,M-SC,15.38,234.0,-1.00,1.14,34.26,OX40N,NTT,PAINTS
50,MASFIN,398.61,1.22,0.12,21.90,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.60,XR,ATH,FINANCE
82,VOLTAS,1530.00,-0.69,7.00,11.87,19.69,24352.0,13413.0,205155.0,-3.43,52.0,X-MC,2.54,78.0,0.55,1.52,14.27,XY25,NTT,AC
43,ITC,452.00,-0.04,-2.28,13.01,10.44,25444.0,-4562.0,195576.0,-41.56,41.0,X-LC,1.93,5.0,-0.18,1.45,3.16,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,12.19,-38.20,118.88,35.27,83425.0,-43373.0,70176.0,-51.71,53.0,L-SC,10.82,270.0,-0.52,0.52,71.76,XR,NTT,HOTELS
0,5PAISA,593.00,1.93,-40.24,88.91,12.90,116100.0,-87914.0,130582.0,92.31,47.0,H-SC,12.03,161.0,-0.76,0.97,8.58,OX40N,NTT,FINANCE
22,DIXON,18931.72,1.52,-12.77,39.38,21.58,48141.0,-17892.0,122247.0,-56.03,40.0,X-MC,8.36,56.0,-0.37,0.91,9.97,X40N,ATH,IT
79,VAIBHAVGBL,521.00,1.34,-26.13,117.65,60.77,158839.0,-47765.0,135010.0,55.92,48.0,H-SC,4.29,159.0,-0.30,1.00,21.59,XR,NTT,JEWELLERY
61,ROUTE,2227.21,1.31,-48.05,225.40,69.05,155817.0,-63937.0,69129.0,-58.05,49.0,H-SC,23.00,140.0,-0.41,0.51,6.61,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,CAMPUS,393.0,-1.71,-12.21,51.83,33.28,76338.0,-20489.0,147286.0,-29.34,36.0,M-SC,2.62,222.0,-0.27,1.09,14.16,XY24,NTT,FOOTWEAR
6,ASIANTILES,137.0,-1.57,-17.07,119.06,81.67,92629.0,-16010.0,77800.0,6988.89,43.0,L-SC,13.01,271.0,-0.17,0.58,50.66,XR,NTT,CERAMICS
16,CERA,9475.0,-1.35,-32.17,82.65,23.89,98611.0,-56591.0,119312.0,-34.61,28.0,H-SC,10.30,157.0,-0.57,0.88,3.04,OX40N,NTT,CERAMICS
9,BAJAJHFL,181.5,-1.12,-22.66,91.94,48.45,142579.0,-45428.0,155078.0,-28.25,22.0,X-MC,12.50,64.0,-0.32,1.15,0.97,X40N,ATH,FINANCE
1,ABB,7934.0,-1.09,-4.22,51.98,45.57,130254.0,-11035.0,250584.0,-38.46,55.0,H-MC,4.60,120.0,-0.08,1.86,6.63,AR,NTT,ELECTRICAL


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,1.22,0.12,21.9,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.6,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.61,-5.68,120.43,107.91,167710.0,-8385.0,139259.0,-23.39,44.0,M-SC,12.67,216.0,-0.05,1.03,3.83,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.88,-11.57,13.11,0.02,20236.0,-20204.0,154355.0,135.49,55.0,M-SC,15.38,234.0,-1.00,1.14,34.26,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.15,-16.61,20.03,0.09,16832.0,-16741.0,84036.0,95.09,43.0,M-SC,2.37,253.0,-0.99,0.62,10.41,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.67,-24.91,51.11,13.47,103498.0,-67167.0,202500.0,-69.09,35.0,H-SC,4.77,158.0,-0.65,1.50,2.97,XY24,NTT,PAINTS
66,SIS,528.00,-0.15,-25.51,62.79,21.26,51933.0,-28323.0,82709.0,1949.22,43.0,H-SC,4.94,166.0,-0.55,0.61,11.75,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.33,6.54,61.69,72.27,111366.0,11078.0,180526.0,-8.35,66.0,M-LC,2.20,99.0,0.10,1.34,13.12,XR,NTT,IT
50,MASFIN,398.61,1.22,0.12,21.90,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.60,XR,ATH,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.33,6.54,61.69,72.27,111366.0,11078.0,180526.0,-8.35,66.0,M-LC,2.20,99.0,0.10,1.34,13.12,XR,NTT,IT
50,MASFIN,398.61,1.22,0.12,21.90,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.60,XR,ATH,FINANCE
38,INDIAMART,4810.62,0.70,-3.41,114.02,106.72,135830.0,-4208.0,119128.0,-52.15,35.0,H-SC,2.69,139.0,-0.03,0.88,19.17,AR,ATH,MISC
25,FINCABLES,1641.55,0.61,-5.68,120.43,107.91,167710.0,-8385.0,139259.0,-23.39,44.0,M-SC,12.67,216.0,-0.05,1.03,3.83,OX40N,ATH,CABLES
37,IEX,219.00,-0.29,-3.10,53.38,48.63,103425.0,-6202.0,193752.0,-35.53,51.0,H-SC,14.03,136.0,-0.06,1.44,8.09,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.12,-22.66,91.94,48.45,142579.0,-45428.0,155078.0,-28.25,22.0,X-MC,12.50,64.0,-0.32,1.15,0.97,X40N,ATH,FINANCE
75,TMPV,600.00,-0.79,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.38,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,0.80,-11.36,28.24,13.67,64348.0,-29203.0,227860.0,-17.95,30.0,X-LC,2.17,14.0,-0.45,1.69,7.57,XY25,NTT,FMCG
53,PAGEIND,51605.08,-0.39,-10.14,40.29,26.07,59283.0,-16600.0,147140.0,-33.76,31.0,X-MC,12.66,55.0,-0.28,1.09,0.25,X40,ATH,APPARELS
51,MEDANTA,1486.00,-0.93,-4.67,30.16,24.08,36500.0,-5930.0,121020.0,-12.39,32.0,X-SC,5.24,89.0,-0.16,0.90,14.03,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-0.40,-80.29,533.08,24.79,264648.0,-202211.0,49645.0,-81.09,43.0,X-SC,0.67,86.0,-0.76,0.37,21.36,X40N,NTT,MISC
43,ITC,452.0,-0.04,-2.28,13.01,10.44,25444.0,-4562.0,195576.0,-41.56,41.0,X-LC,1.93,5.0,-0.18,1.45,3.16,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.69,6.61,22.84,30.97,43037.0,11690.0,188428.0,-18.51,61.0,X-MC,2.10,75.0,0.27,1.40,20.01,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,0.80,-11.36,28.24,13.67,64348.0,-29203.0,227860.0,-17.95,30.0,X-LC,2.17,14.0,-0.45,1.69,7.57,XY25,NTT,FMCG
3,ACC,3906.0,0.86,-24.90,118.58,64.15,211902.0,-59251.0,178700.0,-55.88,39.0,X-SC,2.39,82.0,-0.28,1.32,0.86,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,-0.39,-10.14,40.29,26.07,59283.0,-16600.0,147140.0,-33.76,31.0,X-MC,12.66,55.0,-0.28,1.09,0.25,X40,ATH,APPARELS
75,TMPV,600.00,-0.79,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.38,XY24,NTT,AUTO
3,ACC,3906.00,0.86,-24.90,118.58,64.15,211902.0,-59251.0,178700.0,-55.88,39.0,X-SC,2.39,82.0,-0.28,1.32,0.86,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-1.12,-22.66,91.94,48.45,142579.0,-45428.0,155078.0,-28.25,22.0,X-MC,12.50,64.0,-0.32,1.15,0.97,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.17,-38.16,118.64,35.20,94399.0,-49102.0,79568.0,4.26,34.0,X-SC,15.99,92.0,-0.52,0.59,1.07,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.0,-0.90,-30.05,104.22,42.84,47388.0,-19537.0,45469.0,-54.27,41.0,X-SC,35.95,83.0,-0.41,0.34,2.87,XY24,NTT,MISC
15,CAMS,4762.0,-0.40,-80.29,533.08,24.79,264648.0,-202211.0,49645.0,-81.09,43.0,X-SC,0.67,86.0,-0.76,0.37,21.36,X40N,NTT,MISC
59,RELAXO,1176.0,0.32,-47.37,189.41,52.31,144698.0,-68766.0,76394.0,-43.96,43.0,X-SC,6.69,91.0,-0.48,0.57,2.26,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-0.17,-38.16,118.64,35.20,94399.0,-49102.0,79568.0,4.26,34.0,X-SC,15.99,92.0,-0.52,0.59,1.07,X40,NTT,FOOTWEAR
51,MEDANTA,1486.0,-0.93,-4.67,30.16,24.08,36500.0,-5930.0,121020.0,-12.39,32.0,X-SC,5.24,89.0,-0.16,0.90,14.03,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.26,-12.73,36.67,19.28,110717.0,-44028.0,301928.0,-23.95,62.0,X-LC,4.67,1.0,-0.40,2.24,11.62,X40,ATH,IT
41,INFY,2275.00,-0.07,8.72,42.45,54.87,144407.0,27294.0,340182.0,-15.29,62.0,X-LC,3.56,2.0,0.19,2.52,17.73,X40,BTT,IT
75,TMPV,600.00,-0.79,-18.32,74.06,42.18,167211.0,-50632.0,225778.0,-26.74,29.0,X-LC,6.49,3.0,-0.30,1.67,0.38,XY24,NTT,AUTO
81,VBL,671.64,-0.84,-3.87,41.13,35.67,124881.0,-12218.0,303624.0,-15.30,55.0,X-LC,4.49,4.0,-0.10,2.25,9.45,X40N,ATH,FMCG
43,ITC,452.00,-0.04,-2.28,13.01,10.44,25444.0,-4562.0,195576.0,-41.56,41.0,X-LC,1.93,5.0,-0.18,1.45,3.16,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.57,-17.07,119.06,81.67,92629.0,-16010.0,77800.0,6988.89,43.0,L-SC,13.01,271.0,-0.17,0.58,50.66,XR,NTT,CERAMICS
50,MASFIN,398.61,1.22,0.12,21.90,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.60,XR,ATH,FINANCE
13,BSOFT,831.70,0.10,-22.02,93.35,50.79,102394.0,-30965.0,109688.0,0.12,67.0,H-SC,6.06,151.0,-0.30,0.81,27.71,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.88,-11.57,13.11,0.02,20236.0,-20204.0,154355.0,135.49,55.0,M-SC,15.38,234.0,-1.00,1.14,34.26,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,-0.69,6.61,22.84,30.97,43037.0,11690.0,188428.0,-18.51,61.0,X-MC,2.10,75.0,0.27,1.40,20.01,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.10,-22.02,93.35,50.79,102394.0,-30965.0,109688.0,0.12,67.0,H-SC,6.06,151.0,-0.30,0.81,27.71,XR,ATH,IT
84,WIPRO,420.00,-0.33,6.54,61.69,72.27,111366.0,11078.0,180526.0,-8.35,66.0,M-LC,2.20,99.0,0.10,1.34,13.12,XR,NTT,IT
41,INFY,2275.00,-0.07,8.72,42.45,54.87,144407.0,27294.0,340182.0,-15.29,62.0,X-LC,3.56,2.0,0.19,2.52,17.73,X40,BTT,IT
50,MASFIN,398.61,1.22,0.12,21.90,22.05,21484.0,120.0,98100.0,-13.73,61.0,H-SC,7.22,164.0,0.01,0.73,42.60,XR,ATH,FINANCE
32,HCLTECH,1922.01,-0.22,8.34,15.12,24.73,39631.0,20188.0,262112.0,13.18,63.0,X-LC,6.68,7.0,0.51,1.94,24.85,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.77
1,25,45.63
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.15
MC    30.25
LC    24.58
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.26
X40      22.34
X40N     10.88
XR       10.20
XY25      9.38
AR        9.32
OX40N     7.79
SR        1.02
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.69
X-MC    23.52
X-LC    21.03
M-SC    12.09
X-SC     6.89
H-MC     4.99
L-SC     1.48
M-MC     1.39
M-LC     1.34
H-LC     1.16
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.27,-7.95,42.86
IT,12.82,-16.51,79.51
FINANCE,9.61,-19.60,70.14
ELECTRICAL,6.22,-11.88,52.86
PAINTS,6.08,-12.50,29.21
MISC,6.08,-58.54,123.48
INSURANCE,4.62,-1.47,37.23
PHARMA,4.08,-1.58,34.09
AUTO,3.40,-24.81,78.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3222155.0,21
AR,1315056.0,10
XR,1285550.0,13
X40,1023969.0,14
X40N,981810.0,9
OX40N,739815.0,10
XY25,376239.0,6
SR,282429.0,2
MH,76902.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3644800.0,25
M-SC,1426818.0,15
X-MC,1310308.0,16
X-SC,991224.0,8
X-LC,923938.0,11
H-MC,406023.0,3
L-SC,261976.0,3
M-LC,111366.0,1
H-LC,76769.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1269834.0      6
           AR         900875.0      5
M-SC       XY24       818204.0      6
H-SC       XR         769635.0      7
X-SC       X40N       506881.0      3
X-MC       X40        455907.0      7
           XY24       400142.0      3
X-LC       X40        379509.0      5
H-SC       OX40N      345125.0      4
M-SC       OX40N      308768.0      5
X-SC       XY24       295790.0      3
H-SC       SR         282429.0      2
X-MC       X40N       272672.0      4
X-LC       XY24       243326.0      2
H-MC       AR         211164.0      2
X-LC       X40N       202257.0      2
H-MC       XY24       194859.0      1
X-SC       X40        188553.0      2
X-MC       XY25       181587.0      2
L-SC       XR         176054.0      2
M-SC       XR         173598.0      2
           AR         126248.0      2
M-LC       XR         111366.0      1
X-LC       XY25        98846.0      2
L-SC       OX40N       85922.0      1
H-SC       MH          76902.0      1
H-LC       AR          76769.0      1
M-MC       XY25        57892.0      1
L-MC       XR          54897.0      1
L-LC       XY25        37914.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
